In [57]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [59]:
pretrain_features = {
    "learning_rate": 0.1,
    "batch_size": 256,
    "epochs": 1000,
    "eval_size": 4*256,
    "momentum": 0.005,
    "weight_decay": 0.0001,
}

In [60]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None
    
    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    
#     scaler = StandardScaler()
#     x_pretrain = scaler.fit_transform(x_pretrain)
#     x_train = scaler.transform(x_train)
#     x_test_transed = scaler.transform(x_test)
#     x_test[x_test.columns] = x_test_transed

    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [61]:
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()

In [62]:
class NN(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(1000, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(64, 10),
            nn.BatchNorm1d(10),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(10, 1),
            )

        for m in self.modules():
            if isinstance(m, nn.Linear):    
                nn.init.xavier_uniform_(m.weight)
        
        
    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.seq(x)
        return x
    
    def make_feature(self, x):
        return self.seq[:-4](x)

In [75]:
def make_feature_extractor(x, y, batch_size=256, eval_size=1000):
    """
    This function trains the feature extractor on the pretraining data and returns a function which
    can be used to extract features from the training and test data.

    input: x: np.ndarray, the features of the pretraining set
              y: np.ndarray, the labels of the pretraining set
                batch_size: int, the batch size used for training
                eval_size: int, the size of the validation set
            
    output: make_features: function, a function which can be used to extract features from the training and test data
    """
    # Pretraining data loading
    in_features = x.shape[-1]
    x_tr, x_val, y_tr, y_val = train_test_split(x, y, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    # model declaration
    model = NN()
    model.to(device)
    model.train()
    
    # TODO: Implement the training loop. The model should be trained on the pretraining data. Use validation set 
    # to monitor the loss.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 200
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, y] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, y] in val_loader:
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

    def make_features(x):
        """
        This function extracts features from the training and test data, used in the actual pipeline 
        after the pretraining.

        input: x: np.ndarray, the features of the training or test set

        output: features: np.ndarray, the features extracted from the training or test set, propagated
        further in the pipeline
        """
        model.eval()
        # TODO: Implement the feature extraction, a part of a pretrained model used later in the pipeline.
        x = torch.tensor(x, dtype=torch.float)
        x = x.to(device)
        x = model.make_feature(x).detach().cpu().numpy()
        return x

    return make_features

In [64]:
def make_pretraining_class(feature_extractors):
    """
    The wrapper function which makes pretraining API compatible with sklearn pipeline
    
    input: feature_extractors: dict, a dictionary of feature extractors

    output: PretrainedFeatures: class, a class which implements sklearn API
    """

    class PretrainedFeatures(BaseEstimator, TransformerMixin):
        """
        The wrapper class for Pretraining pipeline.
        """
        def __init__(self, *, feature_extractor=None, mode=None):
            self.feature_extractor = feature_extractor
            self.mode = mode

        def fit(self, X=None, y=None):
            return self

        def transform(self, X):
            assert self.feature_extractor is not None
            X_new = feature_extractors[self.feature_extractor](X)
            return X_new
        
    return PretrainedFeatures

In [76]:
feature_extractor =  make_feature_extractor(x_pretrain, y_pretrain)
PretrainedFeatureClass = make_pretraining_class({"pretrain": feature_extractor})
pretrainedfeatures = PretrainedFeatureClass(feature_extractor="pretrain")

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1: train loss: 4.0283463814793805, val loss: 1.5693160696029662
Epoch 2: train loss: 0.9947594458229688, val loss: 0.5442820992469788
Epoch 3: train loss: 0.3369163450416254, val loss: 0.2153931427001953
Epoch 4: train loss: 0.15638039969059886, val loss: 0.122977448284626
Epoch 5: train loss: 0.10005794372728893, val loss: 0.0792028387784958
Epoch 6: train loss: 0.0832228323525312, val loss: 0.07874281734228133
Epoch 7: train loss: 0.07749177219308152, val loss: 0.06954656332731247
Epoch 8: train loss: 0.07360066453048161, val loss: 0.07211814796924591
Epoch 9: train loss: 0.0709004591131697, val loss: 0.06663256853818894
Epoch 10: train loss: 0.06988233342584299, val loss: 0.06745617401599883
Epoch 11: train loss: 0.068205594691695, val loss: 0.0673437911272049
Epoch 12: train loss: 0.06719756874016353, val loss: 0.06493486413359642
Epoch 13: train loss: 0.06721595986521973, val loss: 0.0623188002705574
Epoch 14: train loss: 0.06653058039533849, val loss: 0.06535713568329811
Ep

In [89]:
from sklearn.linear_model import LinearRegression, ElasticNet
def get_regression_model():
    """
    This function returns the regression model used in the pipeline.

    input: None

    output: model: sklearn compatible model, the regression model
    """
    model = ElasticNet(alpha=0.01, l1_ratio=0.01, max_iter=100000)
    return model

In [66]:
result_file = "results-nn-lr.csv"

In [90]:
pipeline = Pipeline([
    ("featureExtractor", pretrainedfeatures),
    ("scaler", StandardScaler()),
    ("regressor", get_regression_model())
])

pipeline.fit(x_train, y_train)

y_pred = np.zeros(x_test.shape[0])
y_pred = pipeline.predict(x_test.to_numpy())

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv(result_file, index_label="Id")
print(f"Predictions saved to {result_file}, all done!")

Predictions saved to results-nn-lr.csv, all done!
